In [6]:
# Cell 1: All the necessary functions (with updated file path)
import pandas as pd
import numpy as np
import tempfile
import webbrowser
import os
import re

# This dictionary maps shape names to their corresponding D3 symbol index.
SHAPE_MAP = {
    'circle': 0,
    'cross': 1,
    'diamond': 2, # Note: R code used 'losange'
    'square': 3,  # Note: R code used 'rectangle'
    'star': 4,
    'triangle': 5,
    'wye': 6,       # Note: R code used 'y'
}

# The javascript code from patron2.txt
PATRON_2_JS = """
        .on('mouseover.fade', fade(0.1))
        .on('mouseout.tooltip', function() {
          tooltip.transition()
          .duration(100)
          .style('opacity', 0);
        })
        .on('mouseout.fade', fade(1))
        .on('mousemove', function() {
          tooltip.style('left', (d3.event.pageX+50) + 'px')
          .style('top', (d3.event.pageY - 28) + 'px');
        })
                .call(d3.drag()
                      .on('start', dragstarted)
                      .on('drag', dragged)
                      .on('end', dragended))

                ;

  // Label ---------------------------------------
  var texts = svg.selectAll('text.label')
              .data(graph.nodes)
              .enter().append('text')
              .attr('class', 'label')
              .attr('fill', 'black')
              .style("font-size",function(d) {  return 10;  })
              .style('text-anchor', 'middle')
              .text(function(d) {  return d.id;  })
              .on('mouseover.fade', fade(0.1))
              .on('mouseout.fade', fade(1))
              ;

  // Functions------------------
var Layout = 'Force'
var dist = 500
  function ticked(size) {
    graph.nodes.sort(function(x, y){
      return d3.ascending(x.size, y.size);
      })

    if(Layout == 'Circle'){
      graph.nodes.forEach(function(d, i) {
        parts = (2*Math.PI)/graph.nodes.length
        d.x =  width/2 + Math.sin(i*parts)*dist;
        d.y = height/2 + Math.cos(i*parts)*dist;
        })
      }
    if(Layout == 'Linear'){
        graph.nodes.forEach(function(d, i) {
          d.x = i*dist/5 + d.size;
          d.y = height/2;
        })
      }

    if(Layout == "Multilayer"){
      simulation.force('link', d3.forceLink(link)
                        .strength(function(d) { return 0.1; })
        )

        .force('x', d3.forceX().x(function(d) { return 100*Math.random();}))
        .force('y', d3.forceY().y(function(d) { return d.layers*dist;}))
    }

      if(Layout == "Force2"){
         location.reload();
      }


      // Curve
      link.attr('d', function(d) {
        var dx = d.target.x - d.source.x,
            dy = d.target.y - d.source.y,
            dr = Math.sqrt(dx * dx + dy * dy)
        return 'M' + d.source.x + ',' + d.source.y + 'A' + dr + ',' + dr + ' 0 0,1 ' + d.target.x + ',' + d.target.y;
      });

      node.attr('transform', function(d) { return 'translate('+d.x+','+d.y+')'; });

      texts.attr('transform', function(d) { return 'translate('+d.x+','+d.y+')'; });
  }


    function dragstarted(d) {
      if (!d3.event.active) simulation.alphaTarget(0.3).restart()
          d.fx = d.x
          d.fy = d.y
    }

    function dragged(d) {
          d.fx = d3.event.x
          d.fy = d.event.y
    }

    function dragended(d) {
          if (!d3.event.active) simulation.alphaTarget(0)
              d3.select(this).classed("fixed", d.fixed = true);
    }

    const linkedByIndex = {};
    graph.links.forEach(d => {linkedByIndex[`${d.source.index},${d.target.index}`] = 1;});

    function isConnected(a, b) {return linkedByIndex[`${a.index},${b.index}`] || linkedByIndex[`${b.index},${a.index}`] || a.index === b.index;}

    function fade(opacity) {
              return d => {
                  node.style('stroke-opacity', function (o) {
                      const thisOpacity = isConnected(d, o) ? 1 : opacity;
                      this.setAttribute('fill-opacity', thisOpacity);
                          return thisOpacity;
                      });

                  link.style('stroke-opacity', o => (o.source === d || o.target === d ? 1 : opacity));

                                      texts.style('stroke-opacity', function (o) {
                      const thisOpacity = isConnected(d, o) ? 1 : opacity;
                      this.setAttribute('fill-opacity', thisOpacity);
                          return thisOpacity;
                      });

              };}

    function dragsubject() {
      var i,
      x = transform.invertX(d3.event.x),
      y = transform.invertY(d3.event.y),
      dx,
      dy;
      for (i = tempData.nodes.length - 1; i >= 0; --i) {
        node = tempData.nodes[i];
        dx = x - node.x;
        dy = y - node.y;

        if (dx * dx + dy * dy < radius * radius) {

          node.x =  transform.applyX(node.x);
          node.y = transform.applyY(node.y);

          return node;
        }
      }
    }

    function releasenode(d) {
      d.fx = null;
      d.fy = null;
      simulation.alpha(0.5).restart();
      ticked(dist = 500)
    }

  /* Sliders------------------------------------------------------------------------*/
    /*Node spaces------------------------------------------------------------------------*/
     /*For force, linear and circular layout------------------------------------------------------*/
     function changeNodeSpace(size) {
       if(Layout == 'Force'){
         simulation.force('charge', d3.forceManyBody()
                            .strength(function(d) { return forceProperties.charge.strength*size; })
                            .distanceMax(forceProperties.charge.distanceMax)
                            .distanceMin(forceProperties.charge.distanceMin))
      simulation.alpha(0.5).restart();
       }
       if(Layout == 'Linear'){
         var selectedLayout = d3.select(this).property("value")
         ticked(dist = size)
       }
       if(Layout == 'Circle'){
         var selectedLayout = d3.select(this).property("value")
         ticked(dist = size)
       }
       if(Layout == 'Multilayer'){

         simulation.force('link', d3.forceLink(link)
                         .strength(function(d) { return 0; })
         )
         .force('Y', d3.forceY().y(function(d) { return d.y * size; }))

         simulation.alpha(0.5).restart();
       }
     }

     d3.select("#sliderNSpace").on("change", function(d){
               selectedValue = this.value
               changeNodeSpace(selectedValue)
     })

     d3.select("#sliderNSpaceV").on("input", function(d){
               selectedValue = this.value
               changeNodeSpace(selectedValue)
     })

     d3.select("#sliderNSpace2").on("change", function(d){
               selectedValue = this.value
               changeNodeSpace(selectedValue)
     })

     d3.select("#sliderNSpaceV2").on("input", function(d){
               selectedValue = this.value
               changeNodeSpace(selectedValue)
     })

    /*For Multilayer layout------------------------------------------------------------------------*/
     function changeNodeSpaceH(size) {
       if(Layout == 'Multilayer'){

         simulation.force('link', d3.forceLink(link)
                         .strength(function(d) { return 0; })
         )
         .force('X', d3.forceX().x(function(d) { return d.x * size; }))
         simulation.alpha(0.5).restart();
       }
     }

     d3.select("#sliderNSpaceH").on("change", function(d){
               selectedValue = this.value
               changeNodeSpaceH(selectedValue)
     })

     d3.select("#sliderNSpaceVH").on("input", function(d){
               selectedValue = this.value
               changeNodeSpaceH(selectedValue)
     })



    /*Node Size------------------------------------------------------------------------*/
    function changeSize(size) {
              node
              .attr('d', d3.symbol().type( function(d,i) { return d3.symbols[d.shape];} ).size(function(d){return d.size * 100 * size;}))
    }

    d3.select("#sliderSize").on("change", function(d){
              selectedValue = this.value
              changeSize(selectedValue)
    })
    d3.select("#sliderSizeV").on("input", function(d){
              selectedValue = this.value
              changeSize(selectedValue)
    })
    /*Node Width------------------------------------------------------------------------*/
    function changeNwidth(size) {node.attr('stroke-width', function(d) { return d.strokeW * size; })}

    d3.select("#sliderNWidth").on("change", function(d){
              selectedValue = this.value
              changeNwidth(selectedValue)
    })
    d3.select("#sliderNWidthV").on("input", function(d){
              selectedValue = this.value
              changeNwidth(selectedValue)
    })
    /*Node opacity------------------------------------------------------------------------*/
    function changeNop(size) {node.style('stroke-opacity', function(d) { return d.size * size; })}

    d3.select("#sliderNop").on("change", function(d){
              selectedValue = this.value
              changeNop(selectedValue)
    })
    d3.select("#sliderNopV").on("input", function(d){
              selectedValue = this.value
              changeNop(selectedValue)
    })
    /*Label size------------------------------------------------------------------------*/
    function changeLabelSize(size) {texts.style("font-size",function(d) {  return d.size * size;  })}

    d3.select("#sliderLab").on("change", function(d){
              selectedValue = this.value
              changeLabelSize(selectedValue)
    })
    d3.select("#sliderLabV").on("input", function(d){
              selectedValue = this.value
              changeLabelSize(selectedValue)
    })
    /*Links Width ------------------------------------------------------------------------*/
    function changeLwidth(size) {link.style('stroke-width', function(d) { return d.weigth * size; })}

    d3.select("#sliderLWidth").on("change", function(d){
              selectedValue = this.value
              changeLwidth(selectedValue)
    })
    d3.select("#sliderLWidthV").on("input", function(d){
              selectedValue = this.value
              changeLwidth(selectedValue)
    })
      /*Links Opacity------------------------------------------------------------------------*/
      function changeLOpacity(size) {
        link.style('opacity', function(d) { return d.weigth * size; })
      }

      d3.select("#sliderLOpacity").on("change", function(d){
                selectedValue = this.value
                changeLOpacity(selectedValue)
      })
      d3.select("#sliderLOpacityV").on("input", function(d){
              selectedValue = this.value
              changeLOpacity(selectedValue)
    })
        /*Links Multilayer Opacity------------------------------------------------------------------------*/
      function changeLMOpacity(size) {
         link.style('opacity', function(d) { return d.intralayer * size; })
       }

       d3.select("#sliderLOpacityM").on("change", function(d){
                 selectedValue = this.value
                 changeLMOpacity(selectedValue)
       })
       d3.select("#sliderLOpacityMV").on("input", function(d){
               selectedValue = this.value
               changeLMOpacity(selectedValue)
     })

     function changeLMOpacity2(size) {
         link.style('opacity', function(d) { return d.interlayer * size; })
       }

       d3.select("#sliderLOpacityM2").on("change", function(d){
                 selectedValue = this.value
                 changeLMOpacity2(selectedValue)
       })
       d3.select("#sliderLOpacityMV2").on("input", function(d){
               selectedValue = this.value
               changeLMOpacity2(selectedValue)
     })
      /*Links curves------------------------------------------------------------------------*/
      function changeLCurve(size) {
        if(size == 0){
          link.attr('d', function(d) {
              var dx = d.target.x - d.source.x,
                  dy = d.target.y - d.source.y,
                  dr = Math.sqrt(dx * dx + dy * dy)*100;
              return 'M' + d.source.x + ',' + d.source.y + 'A' + dr + ',' + dr + ' 0 0,1 ' + d.target.x + ',' + d.target.y;
            });
        }
        else{
          link.attr('d', function(d) {
            var dx = d.target.x - d.source.x,
                dy = d.target.y - d.source.y,
                dr = Math.sqrt(dx * dx + dy * dy)/size;
            return 'M' + d.source.x + ',' + d.source.y + 'A' + dr + ',' + dr + ' 0 0,1 ' + d.target.x + ',' + d.target.y;
            });
        }
      }

      d3.select("#sliderLCurve").on("change", function(d){
                selectedValue = this.value
                changeLCurve(selectedValue)
      })
      d3.select("#sliderLCurveV").on("input", function(d){
              selectedValue = this.value
              changeLCurve(selectedValue)
    })
      /*Change Layouts------------------------------------------------------------------------*/
      // Create data = list of groups
      layouts = ["Force", "Circle", "Linear", "Multilayer"]

      // Initialize the button
      var dropdownButton = d3.select("#dataviz_builtWithD3")
          .append('select')

      // add the options to the button
      dropdownButton // Add a button
        .selectAll('myOptions') // Next 4 lines add 6 options = 6 colors
             .data(layouts)
        .enter()
            .append('option')
        .text(function (d) { return d; }) // text showed in the menu
        .attr("value", function (d) { return d; }) // corresponding value returned by the button

        // When the button is changed, run the updateChart function
      dropdownButton.on("change", function(d) {

      // recover the option that has been chosen
      var selectedLayout = d3.select(this).property("value")

      // run the updateChart function with this selected option
          if(selectedLayout == 'Circle'){Layout = 'Circle';simulation.alpha(0.5).restart();}
          if(selectedLayout == 'Linear'){Layout = 'Linear';simulation.alpha(0.5).restart();}
          if(selectedLayout == 'Force'){Layout = 'Force2';simulation.alpha(0.5).restart();}
          if(selectedLayout == 'Multilayer'){Layout = 'Multilayer';simulation.alpha(0.5).restart();}

      })
"""

def mat_to_edgl(M, sym=False, erase_diag=True):
    """
    Converts a square adjacency matrix into a DataFrame with three columns 
    representing an edge list. Columns are: 'from', 'to', and 'weight'.
    """
    if not isinstance(M, pd.DataFrame) or M.shape[0] != M.shape[1]:
        raise ValueError("M must be a square pandas DataFrame.")
    if M.columns.empty or M.index.empty:
        raise ValueError("Argument M must have column and index names.")

    if sym:
        mask = np.tril(np.ones(M.shape), k=-1 if erase_diag else 0).astype(bool)
        rows, cols = np.where(mask)
        df = pd.DataFrame({
            'from': M.columns[cols],
            'to': M.index[rows],
            'weight': M.values[rows, cols]
        })
    else:
        M_copy = M.copy()
        M_copy.index.name = 'to'
        M_copy.columns.name = 'from'

        df = M_copy.stack().reset_index(name='weight')
        df = df[['from', 'to', 'weight']]

        if erase_diag:
            df = df[df['from'] != df['to']]
    
    return df.reset_index(drop=True)


def df_col_find_id(df, label_name):
    """Finds the data frame column name from the name of the column or its index."""
    if not isinstance(label_name, (str, int)):
        raise TypeError("Argument label_name must be a string or an integer.")

    if isinstance(label_name, str):
        if label_name not in df.columns:
            raise ValueError(f"Column name '{label_name}' not found in DataFrame.")
        return label_name
    else:
        if label_name >= len(df.columns) or label_name < 0:
            raise IndexError("Column index is out of bounds.")
        return df.columns[label_name]


def _hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))


def _rgb_to_hex(rgb_color):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb_color[0]), int(rgb_color[1]), int(rgb_color[2]))


def colorize(df, col_att, color, new_col_name):
    col_name = df_col_find_id(df, col_att)
    sorted_unique_vals = sorted(df[col_name].unique())
    n_colors = len(sorted_unique_vals)
    start_rgb = _hex_to_rgb(color[0])
    end_rgb = _hex_to_rgb(color[1])
    palette = [_rgb_to_hex([np.interp(i, [0, n_colors - 1], [start_rgb[j], end_rgb[j]]) for j in range(3)]) for i in range(n_colors)]
    color_map = dict(zip(sorted_unique_vals, palette))
    df_copy = df.copy()
    df_copy[new_col_name] = df_copy[col_name].map(color_map)
    return df_copy


def shape(vec, char):
    unique_vec = vec.unique()
    if len(unique_vec) > 7:
        raise ValueError("We don't have more than 7 node shapes to propose. Sorry...")
    if len(unique_vec) != len(char):
        raise ValueError("Argument 'char' must be of the same length as the number of unique values in 'vec'.")
    shape_mapping = {val: SHAPE_MAP[shape_name] for val, shape_name in zip(unique_vec, char)}
    numeric_shapes = vec.map(shape_mapping)
    if numeric_shapes.isnull().any():
        raise ValueError("One of the symbols declared in argument 'char' is not supported.")
    return numeric_shapes.astype(int)


def vis_net_format_att(df, col_id=None, col_size=None, color=None, col_color=None, shapes=None, col_shape=None, strokeCol=None, col_strokeCol=None, col_stroke=None, node_opacity=None):
    df = df.copy()
    if any(arg is not None for arg in [color, strokeCol, col_shape, node_opacity]) and col_id is None:
        raise ValueError("Argument col_id cannot be None if other arguments are not.")
    ori_cols = {}
    if node_opacity is not None:
        opacity_col_name = df_col_find_id(df, node_opacity)
        if pd.api.types.is_numeric_dtype(df[opacity_col_name]):
            ori_cols['opacity'] = opacity_col_name
            min_op, max_op = df[opacity_col_name].min(), df[opacity_col_name].max()
            df['opacity'] = 1.0 if max_op - min_op == 0 else (df[opacity_col_name] - min_op) / (max_op - min_op)
            df.loc[df['opacity'] == 0, 'opacity'] = 0.001
        else:
            raise TypeError("Column for argument node_opacity needs to be numeric.")
    else:
        ori_cols['opacity'] = None
        df['opacity'] = 1.0
    if col_size is not None:
        size_col_name = df_col_find_id(df, col_size)
        if pd.api.types.is_numeric_dtype(df[size_col_name]):
            ori_cols['size'] = size_col_name
            df['size'] = df[size_col_name]
        else:
            raise TypeError("Column for argument col_size needs to be numeric.")
    else:
        ori_cols['size'] = None
        df['size'] = 1
    if col_id is not None:
        id_col_name = df_col_find_id(df, col_id)
        ori_cols['id'] = id_col_name
        df['id'] = df[id_col_name]
    else:
        ori_cols['id'] = None
        df['id'] = range(len(df))
    if col_shape is not None:
        shape_col_name = df_col_find_id(df, col_shape)
        ori_cols['shape'] = shape_col_name
        df['shape'] = shape(df[shape_col_name], shapes) if shapes else pd.factorize(df[shape_col_name])[0]
    else:
        ori_cols['shape'] = None
        df['shape'] = 0
    if col_stroke is not None:
        stroke_col_name = df_col_find_id(df, col_stroke)
        if pd.api.types.is_numeric_dtype(df[stroke_col_name]):
            ori_cols['stroke'] = stroke_col_name
            df['strokeW'] = df[stroke_col_name]
        else:
            raise TypeError("Column for argument col_stroke needs to be numeric.")
        if col_strokeCol is not None:
            if strokeCol and len(strokeCol) == 2:
                stroke_color_col_name = df_col_find_id(df, col_strokeCol)
                ori_cols['strokeCol'] = stroke_color_col_name
                df = colorize(df, stroke_color_col_name, strokeCol, new_col_name='strokeCol')
            else:
                raise ValueError('Argument strokeCol must be a list of length 2 to build a gradient.')
        else:
            ori_cols['strokeCol'] = None
            df['strokeCol'] = 'white'
    else:
        ori_cols.update({'strokeCol': None, 'stroke': None})
        df['strokeCol'] = 'rgba(0,0,0,0)'
        df['strokeW'] = 0
    if col_color is not None:
        if color and len(color) == 2:
            color_col_name = df_col_find_id(df, col_color)
            ori_cols['color'] = color_col_name
            df = colorize(df, col_att=color_col_name, color=color, new_col_name='color')
        else:
            raise ValueError('Argument color must be a list of length 2 to build a gradient.')
    else:
        ori_cols['color'] = None
        df['color'] = 'black'
    df['sizeValue'] = df.get(ori_cols.get('size'), df['size'])
    df['colorValue'] = df.get(ori_cols.get('color'), 'N/A')
    df['strokeColValue'] = df.get(ori_cols.get('strokeCol'), 'N/A')
    df['strokeWValue'] = df.get(ori_cols.get('stroke'), df['strokeW'])
    df['shapeValue'] = df.get(ori_cols.get('shape'), 'N/A')
    ori_list = [ori_cols.get(k) for k in ['id', 'size', 'color', 'strokeCol', 'stroke', 'shape', 'opacity']]
    return df, ori_list


def vis_net(df, m, col_id=None, col_size=None, color=('black', 'white'), col_color=None, col_shape=None, shapes=None, strokeCol=('white', 'black'), col_strokeCol=None, col_stroke=None, layers=None, node_opacity=None, link_opacity=False, background='grey'):
    """
    Visualizes a network by generating an HTML file with D3.js.
    Opens the default web browser to display the network.
    """
    if df is None:
        raise ValueError("DataFrame 'df' cannot be None.")

    d, ori = vis_net_format_att(df, col_id=col_id, col_size=col_size, color=color, col_color=col_color, col_shape=col_shape, shapes=shapes, strokeCol=strokeCol, col_strokeCol=col_strokeCol, col_stroke=col_stroke, node_opacity=node_opacity)
    df = d

    if ori[0] is None:
        df['id'] = m.columns.tolist()
    
    df['layers'] = pd.factorize(df[layers])[0] + 1 if layers else 1

    edgl = mat_to_edgl(m)
    edgl = edgl[edgl['weight'] != 0].copy()

    if link_opacity:
        min_w, max_w = edgl['weight'].min(), edgl['weight'].max()
        edgl['lOpacity'] = 1.0 if max_w - min_w == 0 else (edgl['weight'] - min_w) / (max_w - min_w)
        edgl.loc[edgl['lOpacity'] == 0, 'lOpacity'] = 0.0001
    else:
        edgl['lOpacity'] = 1.0

    tmp = df[['id', 'color']].rename(columns={'id': 'from'})
    edgl = pd.merge(edgl, tmp, on='from', how='left').rename(columns={'color': 'colorL'})
    
    if layers:
        id_to_layer_map = df.set_index('id')['layers'].to_dict()
        test1 = edgl['from'].map(id_to_layer_map)
        test2 = edgl['to'].map(id_to_layer_map)
        edgl['intralayer'] = (test1 == test2).astype(int)
        edgl['interlayer'] = np.nan
        edgl.loc[edgl['intralayer'] == 0, 'interlayer'] = 1
        edgl.loc[edgl['intralayer'] == 1, 'intralayer'] = 1
        edgl.loc[edgl['interlayer'] != 1, 'intralayer'] = np.nan
    else:
        edgl['intralayer'] = np.nan
        edgl['interlayer'] = np.nan

    # --- CHANGE IS HERE ---
    # Save the file in the current working directory.
    html_file_path = 'NetExplorer.html'
    
    with open(html_file_path, 'w', encoding='utf-8') as f:
        f.write(f'<!DOCTYPE html><meta charset="utf-8"><style>body{{background-color:{background};}}.links line{{stroke:#999;stroke-opacity:0.6;}}.nodes circle{{stroke:#fff;stroke-width:1.5px;}}.tooltip{{position:absolute;text-align:center;width:auto;height:auto;padding:8px;font:12px sans-serif;background:lightsteelblue;border:0px;border-radius:8px;pointer-events:none;}}</style><svg width="1800" height="1200"></svg><div id="dataviz_builtWithD3"></div><script src="https://d3js.org/d3.v4.min.js"></script><script src="https://d3js.org/d3-symbol.v1.min.js"></script><script>var svg=d3.select("svg"),width=+svg.attr("width"),height=+svg.attr("height");var graph=getData();var forceProperties={{center:{{x:0.5,y:0.5}},charge:{{enabled:true,strength:-100,distanceMin:1,distanceMax:2000}},link:{{enabled:true,distance:30,strength:1}}}};var simulation=d3.forceSimulation().force("link",d3.forceLink().id(function(d){{return d.id;}})).force("charge",d3.forceManyBody().strength(forceProperties.charge.strength)).force("center",d3.forceCenter(width/2,height/2));var link=svg.append("g").attr("class","links").selectAll("path").data(graph.links).enter().append("path").style("stroke-width",function(d){{return d.weigth;}}).style("stroke",function(d){{return d.colorL;}}).style("opacity",function(d){{return d.lOpacity;}});var tooltip=d3.select("body").append("div").attr("class","tooltip").style("opacity",0);var node=svg.append("g").attr("class","nodes").selectAll("path").data(graph.nodes).enter().append("path").attr("d",d3.symbol().type(function(d){{return d3.symbols[d.shape];}}).size(function(d){{return d.size*100;}})).attr("fill",function(d){{return d.color;}}).style("stroke",function(d){{return d.strokeCol;}}).style("stroke-width",function(d){{return d.strokeW;}}).on("mouseover.tooltip",function(d){{tooltip.transition().duration(300).style("opacity",0.9);tooltip.html(')
        
        tooltip_html = "'id : ' + d.id "
        if ori[1]: tooltip_html += f" + '<p/>{ori[1]} (size) : ' + d.sizeValue"
        if ori[2]: tooltip_html += f" + '<p/>{ori[2]} (color) : ' + d.colorValue"
        if ori[3]: tooltip_html += f" + '<p/>{ori[3]} (stroke color) : ' + d.strokeColValue"
        if ori[4]: tooltip_html += f" + '<p/>{ori[4]} (stroke width) : ' + d.strokeWValue"
        if ori[5]: tooltip_html += f" + '<p/>{ori[5]} (shape) : ' + d.shapeValue"
        f.write(tooltip_html)

        f.write(').style("left",(d3.event.pageX)+"px").style("top",(d3.event.pageY+10)+"px");})')
        f.write(PATRON_2_JS)
        f.write("\nfunction getData() {\n   let json = { 'nodes':[\n")

        nodes_json_cleaned = re.sub(r'"(\w+)":', r'\\1:', df.to_json(orient='records'))
        f.write(nodes_json_cleaned[1:-1])
        
        f.write("\n],'links':[\n")
        
        edgl.rename(columns={'weight': 'weigth'}, inplace=True)
        links_json_cleaned = re.sub(r'"(\w+)":', r'\\1:', edgl.to_json(orient='records'))
        f.write(links_json_cleaned[1:-1])

        f.write("\n]};\nreturn json;\n}\n</script>\n</body>\n</html>")

    webbrowser.open('file://' + os.path.realpath(html_file_path))
    print(f"Generated HTML file at: {os.path.realpath(html_file_path)}")

In [7]:
# Cell 2: Test script to generate data and call the function

def create_test_data():
    """Creates a sample DataFrame and adjacency matrix for testing."""
    node_data = {
        'id': ['A', 'B', 'C', 'D', 'E'],
        'strength': [10, 20, 15, 25, 18],
        'age': [25, 30, 22, 40, 35],
        'kinship': ['Family', 'Friend', 'Family', 'Work', 'Friend'],
        'degree': [2, 3, 1, 4, 2],
        'sex': ['M', 'F', 'M', 'M', 'F'],
        'centrality': [0.8, 0.5, 0.9, 0.4, 0.7]
    }
    node_attributes = pd.DataFrame(node_data)
    nodes = node_attributes['id']
    adjacency_matrix = pd.DataFrame(
        np.array([
            [0, 5, 2, 0, 0],
            [5, 0, 0, 1, 3],
            [2, 0, 0, 0, 0],
            [0, 1, 0, 0, 4],
            [0, 3, 0, 4, 0]
        ]),
        index=nodes,
        columns=nodes
    )
    return node_attributes, adjacency_matrix

# Generate the sample data and run the visualization
node_attributes_df, adjacency_matrix_m = create_test_data()

print("--- Node Attributes (df) ---")
print(node_attributes_df)
print("\n--- Adjacency Matrix (m) ---")
print(adjacency_matrix_m)
print("\nGenerating visualization... Your browser should open shortly.")

vis_net(
    df=node_attributes_df,
    m=adjacency_matrix_m,
    col_id="id",
    col_size="strength",
    color=['#d9f0a3', '#00441b'],
    col_color="age",
    strokeCol=['#feebe2', '#7a0177'],
    col_strokeCol="kinship",
    col_stroke="degree",
    col_shape="sex",
    shapes=['circle', 'triangle'],
    layers="kinship",
    node_opacity="centrality",
    link_opacity=True,
    background="lightgrey"
)

--- Node Attributes (df) ---
  id  strength  age kinship  degree sex  centrality
0  A        10   25  Family       2   M         0.8
1  B        20   30  Friend       3   F         0.5
2  C        15   22  Family       1   M         0.9
3  D        25   40    Work       4   M         0.4
4  E        18   35  Friend       2   F         0.7

--- Adjacency Matrix (m) ---
id  A  B  C  D  E
id               
A   0  5  2  0  0
B   5  0  0  1  3
C   2  0  0  0  0
D   0  1  0  0  4
E   0  3  0  4  0

Generating visualization... Your browser should open shortly.
Generated HTML file at: /home/sosa/work/BI/Test/NetExplorer.html


Try running the update-desktop-database command. If you
don't have this command you should install the
desktop-file-utils package. This package is available from
http://freedesktop.org/wiki/Software/desktop-file-utils/
No applications found for mimetype: text/html
./usr/bin/xdg-open: 882: x-www-browser: not found
/usr/bin/xdg-open: 882: firefox: not found
/usr/bin/xdg-open: 882: iceweasel: not found
/usr/bin/xdg-open: 882: seamonkey: not found
/usr/bin/xdg-open: 882: mozilla: not found
/usr/bin/xdg-open: 882: epiphany: not found
/usr/bin/xdg-open: 882: konqueror: not found
/usr/bin/xdg-open: 882: chromium: not found
/usr/bin/xdg-open: 882: chromium-browser: not found
/usr/bin/xdg-open: 882: google-chrome: not found
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method avai